## PERIN

Preprocess the data from the stancer pipeline and bring it into canonical form so it can be used by the sentiment graph parser.

In [1]:
# IMPORTS
import pandas as pd
import json
import os

In [2]:
# makedirs if not exist
os.makedirs("../../etl/data/processed/Perin_Preprocessing", exist_ok=True)

In [3]:
TRAIN_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_train.csv"
TEST_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_test.csv"
VAL_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_val.csv"

TRAIN_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_train.json"
TEST_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_test.json"
VAL_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_val.json"

MANANN_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/manual_annotation.csv"
MANANN_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/manual_annotation.json"

# Full phrase or Head mode
FULL_PHRASE = False
USE_FULL_SENTENCE = True

In [4]:
# read in each dataframe
input_paths = [TRAIN_INPUT_PATH, TEST_INPUT_PATH, VAL_INPUT_PATH, MANANN_INPUT_PATH]
dataframes = []

for ip in input_paths:
    df = pd.read_csv(ip, index_col=0)
    dataframes.append(df)


In [5]:
# inspect a df
dataframes[1]

,verb_form,verb_form_start,verb_form_end,verb_lemma,arg1,arg1_start,arg1_end,arg1_pos,arg1_head,arg1_head_start,...,arg2,arg2_start,arg2_end,arg2_pos,arg2_head,arg2_head_start,arg2_head_end,rel_type,pred_serial,full_sentence_text
doc_id,,,,,,,,,,,,,,,,,,,,,
f0c73a8ed844a853e6f68d6ad1889e8983e65fc23ffaa67fa4758daec7514e2b,abgetan,141,148,abtun,der weisse Quarterback Drew Brees,8,41,N,Quarterback,19,...,die Proteste von Spielern,69,94,N,Proteste,73,81,con,"Predicate(type='con', args=(Head(sentence=47, ...",Nachdem der weisse Quarterback Drew Brees von ...
1aa7eb9f3bfdc6080786b612e1868877b17b08b0cc8f0fbe9fb9e7813acb746e,verschlimmert,68,81,verschlimmern,Die Pandemie,0,12,N,Pandemie,4,...,die bereits ohnehin schwierige Situation,18,58,N,Situation,49,58,con,"Predicate(type='con', args=(Head(sentence=13, ...",Die Pandemie habe die bereits ohnehin schwieri...
e6a78f157b883206b60e5f8296782be195cced85d7d331d7ec9e45ce1d33df05,abbrechen,85,94,abbrechen,Auch der linke Präsidentschaftskandidat Gustav...,0,53,N,Präsidentschaftskandidat,15,...,einen Wahlkampfauftritt,61,84,N,Wahlkampfauftritt,67,84,con,"Predicate(type='con', args=(Head(sentence=12, ...",Auch der linke Präsidentschaftskandidat Gustav...
1ca44ad3dbe7df0e4148d4b14f0d2a310696605616a5babbb57233db7aae4181,fernhalten,123,133,fernhalten,das IOK 28Russen,84,100,FM,IOK,51,...,den Winterspielen,105,122,N,Winterspielen,109,122,con,"Predicate(type='con', args=(Head(sentence=14, ...",Im Januar 2018 hatte das CAS bereits einen and...
c0d6e151150fa54623830727388202a8a0a990522066fc6868a7098602c59645,angezeigt,54,63,anzeigen,Die Polizei,9,20,N,Polizei,13,...,einen 26-jährigen Autofahrer angezeigt,25,64,N,Autofahrer,43,53,con,"Predicate(type='con', args=(Head(sentence=5, t...",Freiburg Die Polizei hat einen 26-jährigen Aut...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a0afd3d30773293bd3e2c55158b5cf3f151332214327f3b3d181e450c086f70b,angekündigt,94,105,ankündigen,die SBB und Bombardier,31,53,N,SBB,35,...,den neuen FV-Dosto,54,72,N,FV-Dosto,64,72,pro,"Predicate(type='pro', args=(Head(sentence=3, t...",Mit Pauken und Trompeten haben die SBB und Bom...
b0604c4fe8946014956b9fa019b40020fa3e0a4896ca3c0cb3b90e5c9a1d5ffc,ergötzte,93,101,ergötzen,Hans Christian Andersen,69,92,N,Hans,69,...,der Blauen Grotte,110,127,N,Grotte,121,127,pro,"Predicate(type='pro', args=(Head(sentence=34, ...",Schwedische Königinnen allerdings verbrachten ...
f77cdb308e72790498167a2f9c032b29c877060caf778cc04d4a85727c939288,stellt,32,38,stellen,ein Reporter,39,51,N,Reporter,43,...,Recherchen,52,62,N,Recherchen,52,62,pro,"Predicate(type='pro', args=(Head(sentence=27, ...",Nach dem Tod eines Medienmoguls stellt ein Rep...


In [6]:
dataframes[1].rel_type.value_counts()

neutral    1792
con        1775
pro        1757
Name: rel_type, dtype: int64

In [7]:
def transform_to_semeval_ssa_format(df):
    sent_list = []
    sentences = list(set(df.full_sentence_text.to_list()))
    for i, sentence in enumerate(sentences):
        opinions_list = []
        sub_df = df[df["full_sentence_text"] == sentence]
        for row in sub_df.itertuples():
            # skip neutral case, not needed here
            # if row.rel_type == "neutral":
            #    continue

            if FULL_PHRASE:
                # only-NPs
                opinions_list.append({
                    "Source": [[row.arg1], [f"{row.arg1_start}:{row.arg1_end}"]],
                    "Target": [[row.arg2], [f"{row.arg2_start}:{row.arg2_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else "Neutral",
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })
            else:
                # non-NPs
                opinions_list.append({
                    "Source": [[row.arg1_head], [f"{row.arg1_head_start}:{row.arg1_head_end}"]],
                    "Target": [[row.arg2_head], [f"{row.arg2_head_start}:{row.arg2_head_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else "Neutral",
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })

        if len(opinions_list) != 0:
            sent_list.append({
                "sent_id": str(i),
                "text": sentence,
                "opinions": opinions_list
            })
        else:
            continue
    return sent_list

In [8]:
# test the function
data = transform_to_semeval_ssa_format(dataframes[1])

In [9]:
# verify data
data[:1]

[{'sent_id': '0',
  'text': 'Andere Proteste entstanden wegen der seit Jahren miserablen Wirtschaftslage und der allumfassenden Korruption im Land und blieben meist regional .',
  'opinions': [{'Source': [['Proteste'], ['7:15']],
    'Target': [['Land'], ['113:117']],
    'Polar_expression': [['entstanden'], ['16:26']],
    'Polarity': 'Neutral',
    'Intensity': 'Average'}]}]

In [10]:
FULL_PHRASE

False

In [11]:
x = "Und auch die unabhängigen Verwaltungsräte sowie Konzernleitung und Management stellten sich quer und rebellierten gegen die unerwünschte Übernahme ."
eval('x[' + '26:41' + ']')

'Verwaltungsräte'

## Minimal Sequence Dataset

The idea here is to see whether a pruning approach, that is learning to make the sentence shorter through syntactic dependency reduction would yield an improved result.

This is, would learning to prune help? https://aclanthology.org/D18-1244/

In [12]:
import copy

def convert_ssa_to_minimal_sequence_data(data):
    data = copy.deepcopy(data)
    ret_data = []
    for sent in data:
        opinions = []
        minimal_sent = ""
        for opinion in sent["opinions"]:
            try:
                # print("OsS", opinion["Source"][1][0])
                minimal_sent += eval('sent["text"][' + opinion["Source"][1][0] + ']') + " "
                # print(opinion["Source"])
                opinion["Source"][1][0] = str(minimal_sent.find(opinion["Source"][0][0])) + ":" + str(len(opinion["Source"][0][0]))

                minimal_sent += eval('sent["text"][' + opinion["Target"][1][0] + ']') + " "
                # print(eval('sent["text"][' + opinion["Target"][1][0] + ']') + " ")
                # print(minimal_sent)
                # print(opinion["Target"])
                opinion["Target"][1][0] = str(minimal_sent.find(opinion["Target"][0][0])) + ":" + str(len(opinion["Target"][0][0]))

                minimal_sent += eval('sent["text"][' + opinion["Polar_expression"][1][0] + ']') + " "
                opinion["Polar_expression"][1][0] = str(minimal_sent.find(opinion["Polar_expression"][0][0])) + ":" + str(len(opinion["Polar_expression"][0][0]))
                opinions.append(opinion)
            except:
                continue
        if len(opinions) > 0:
            sent["opinions"] = opinions
            # add a dot to every sentence at the end, possibly fixes problem
            sent["text"] = minimal_sent + "."
            ret_data.append(sent)
        else:
            continue
    return ret_data

In [13]:
# data_minimal

In [14]:
output_paths = [TRAIN_OUTPUT_PATH, TEST_OUTPUT_PATH, VAL_OUTPUT_PATH, MANANN_OUTPUT_PATH]

for op, df in zip(output_paths, dataframes):
    # call function for each... dev, test, train-json
    data = transform_to_semeval_ssa_format(df)
    # convert to JSON format.
    if not USE_FULL_SENTENCE:
        data = convert_ssa_to_minimal_sequence_data(data)
    with open(op, mode="w") as f:
        json.dump(data, f, ensure_ascii=False)

In [15]:
USE_FULL_SENTENCE

True